In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [6]:
df_transactions = pd.read_csv(x + "Thalia_transactions_ersteBereinigung.csv")

In [7]:
df_privatkunden_iids = pd.read_csv(x +
                                    "privatkunden.iid.thalia.csv", index_col = [0])

##### Privatkunden und Trans mergen

In [8]:
# Privatkunden iids mit Transaktionsdaten mergen
df_t = pd.merge(df_transactions, df_privatkunden_iids, on = ["iid"])
df_t.head()

,iid,date,month_count
0,100002747,2022-02-03 12:44:22,21
1,100002747,2022-04-29 14:03:00,23
2,100009615,2018-09-10 09:52:00,-20
3,100009615,2018-10-22 10:32:00,-19
4,100011333,2022-05-27 10:24:33,24


##### Auswahl der Kunden: Transaktion im November 2021 und mindestes eine Transaktion nach Juni 2022

In [9]:
# Alle "iid" mit "month_count" = 1
iid_month_1 = df_t.groupby("iid").filter(lambda x: (x["month_count"] == 18).any())["iid"].unique()

# Alle "iid" mit "month_count" >= 8
iid_month_8 = df_t.groupby("iid").filter(lambda x: (x["month_count"] >= 24).any())["iid"].unique()

# Die Schnittmenge beider Listen gibt alle "iid" aus, die beide Bedingungen erfüllen
iid_both = list(set(iid_month_1) & set(iid_month_8))

##### Mergen der Auswahl mit Trans_df

In [10]:
# Annahme: iid_both ist eine Liste von eindeutigen iids
df_t_filtered = df_t[df_t['iid'].isin(iid_both)]

##### Filter nach Trans nach t > 17

In [11]:
# Ausschließlich Transaktionen nach t > -1 relevant
df_t_filtered = df_t_filtered[df_t_filtered["month_count"] > 17]

#### Speicherung

In [12]:
path = x
filename = 'Thalia_transactions_zweiteBereinigung_Validierung.csv'

df_t_filtered.to_csv(os.path.join(path, filename), index=False)

In [13]:
path = x
filename = 'Thalia_transactions_zweiteBereinigung_iids_Validierung.csv'

df_iid_both = pd.DataFrame(iid_both, columns=["iid"])
df_iid_both.to_csv(os.path.join(path, filename), index=False)